# Load cases for each configuration

In [4]:
import sys
import numpy as np
sys.path.append('../')

# from Material import Material
from Equilibrium import PointLoad, RunningLoad, EquilibriumEquation, Moment
# from Geometry import WingBox
from cg_est import Wing, Fuselage, LandingGear, Propulsion, Weight
from MathFunctions import StepFunction, Poly as Polynomial
from constants import *

In [10]:
My = StepFunction([[1, 2, 3], [4, 5, 6]], 'y')
y2 = Polynomial([1, 2, 3], symbol='x')
show(y2)
show(y2*3)


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
from IPython.display import Math, display
show = lambda func: display(Math(func.__str__()))
SNCurve = lambda Sa, m=4.1, C=3.15e14: C * Sa ** (-m)

**Eliptical Lift Distribution**

$(z/b)^2 + (y/a)^2 = 1$

$2 L / (\pi \cdot b) = a$

$y = a \cdot \sqrt{1 - 4 \cdot (z/b)^2}$

**Equilibrium**

$\sum \vec{F} = \vec{0}$

$\sum \vec{M} = \vec{0}$

**Shear**

$q_{2} - q_1 = \int_{s1}^{s2}\frac{\delta q}{\delta s} ds = -\frac{V_y I_{yy} + V_x I_{xy}}{I_{xx}I_{yy} + I_{xy}^2}\int_{s1}^{s2}tyds - \frac{V_x I_{xx} + V_y I_{xy}}{I_{xx}I_{yy} + I_{xy}^2}\int_{s1}^{s2}txds$

**Bending**

$\sigma_z = \frac{M_x y}{I_{xx}} + \frac{M_y x}{I_{yy}}$

## Tandem Wing

In [ ]:
config = 1

#sensitivity ------------
#AR = 14
#------------------------


WoS = WS
Pmax = 8.77182
mProp = 400 / N_cruise
thickness = 3e-3
w_fus= 1.3; h_fus=1.6; l_fus=4
nmax = 3.02
b = (AR * S_front) ** 0.5

w = Weight(95, Wing(mTO, S_front, S_back, 1.5*nmax, AR, [0.4, 3.6], config),
           Fuselage(mTO, Pmax, l_fus, 5, l_fus/2, config),
          LandingGear(mTO, l_fus/2),
          Propulsion(N_cruise, [mProp]*N_cruise, pos_prop=[3.6]*int(N_cruise/2) + [0.4]*int(N_cruise/2)),
          cargo_m, cargo_p, Bat_mass, l_fus/2, [0.8, 1.3, 1.3, 2.5, 2.5])

ukloads = [PointLoad([1, 0, 0], [0, 0, 0]), PointLoad([0, 1, 0], [0, 0, 0]), PointLoad([0, 0, 1], [0, 0, 0]),
          Moment([1, 0, 0]), Moment([0, 1, 0]), Moment([0, 0, 1])]

Thrusts = [PointLoad([0, Max_T_engine, 0], [0, 0, d]) for d in np.linspace(0, b/2, round(N_hover / 4))]
distrWeight = RunningLoad([[0]*3, [ -w.wing.get_weight()[0] * 9.81 / b ]*3], [0, b/4, b/2], 2)


aluminum = Material.load(material='Al 6061', Condition='T6')
box = WingBox(thickness, 0.8 * 0.17 * c_r, 0.8 * c_r, aluminum)

eql = EquilibriumEquation(kloads=[distrWeight], ukloads=ukloads)
eql.SetupEquation()
Fx, Fy, Fz, Mx, My, Mz = eql.SolveEquation()

print("AR = ", AR, " b = ", b)

print("VTOL: τ, σ, Y [MPa]")
print(tauVTOL := box.tau(box.b/2, 0, Fx, Fy, Mz)*1e-6)
print(oVTOL := box.o(-box.b/2, -box.h/2, -Mx, My)*1e-6)
print(YVTOL := (oVTOL ** 2 + 3 * tauVTOL ** 2) ** 0.5)

rho = 1.205
toc = 0.17
box = WingBox(thickness, 0.8 * c_r, 0.8 * 0.17 * c_r, aluminum)

liftWing = 2 * (9.81 * mTO * nmax * 1.5 / 2) / ( np.pi * b )
zpos = np.linspace(0, b/2, 1000)
Lift = RunningLoad([[0] * len(zpos), [liftWing * ( 1 - (z/b) ** 2 ) ** 0.5 for z in zpos]], zpos, axis=2)
Drag = RunningLoad([[liftWing / LD_ratio * ( 1 - (z/b) ** 2 ) ** 0.5 for z in zpos], [0] * len(zpos)], zpos, axis=2, poa=(-0.25 * MAC1, 0))
ADMoment = Moment([0, 0, Cm_ac_front * 0.5 * rho * V_cruise ** 2 * S_front * MAC1])
Thrusts = [PointLoad([-liftWing * 4 / (LD_ratio*N_cruise), 0, 0],
                     [0.45 * MAC1, toc * MAC1 * 0.5, d]) for d in np.linspace(0, b/2, round(N_cruise / 4))]

cruise = EquilibriumEquation(kloads=[Lift, Drag, ADMoment, distrWeight] + Thrusts,
                            ukloads=ukloads)
cruise.SetupEquation()
Rfx, Rfy, Rfz, Rmx, Rmy, Rmz = cruise.SolveEquation()
print("\nCruise: τ, σ, Y [MPa]")
print(taucr := box.tau(-box.b/2, 0, Rfx, Rfy, Rmz)*1e-6)
print(ocr := box.o(-box.b/2, -box.h/2, -Rmx, Rmy)*1e-6)
print(Ycr := (ocr ** 2 + 3 * taucr ** 2) ** 0.5)

print("\nFatigue:")
print(fatigueLife := aluminum.ParisFatigueN(ocr*1e6 - oVTOL*1e6, box.b, 0.375 * 1.2e-3, box.t/2)*1e-6)
print(bucklingStress := aluminum.buckling(box.b, box.t)*1e-6)
print(snLife := SNCurve(ocr - oVTOL)*1e-6)



## Box Wing

In [ ]:
config = 2

#sensitivity ------------
#AR = 7
#------------------------


WoS = WS
Pmax = 8.77182
mProp = 400 / N_cruise
thickness = 3e-3
w_fus= 1.3; h_fus=1.6; l_fus=4
nmax = 3.02
b = (AR * S_front) ** 0.5

aluminum = Material.load(material='Al 6061', Condition='T6')

w = Weight(95, Wing(mTO, S_front, S_back, 1.5*nmax, AR, [0.4, 3.6], config),
           Fuselage(mTO, Pmax, l_fus, 5, l_fus/2, config),
          LandingGear(mTO, l_fus/2),
          Propulsion(N_cruise, [mProp]*N_cruise, pos_prop=[3.6]*int(N_cruise/2) + [0.4]*int(N_cruise/2)),
          cargo_m, cargo_p, Bat_mass, l_fus/2, [0.8, 1.3, 1.3, 2.5, 2.5])

ukloads = [PointLoad([1, 0, 0], [0, 0, 0]), PointLoad([0, 1, 0], [0, 0, 0]), PointLoad([0, 0, 1], [0, 0, 0]),
          Moment([1, 0, 0]), Moment([0, 1, 0]), Moment([0, 0, 1])]

Thrusts = [PointLoad([0, Max_T_engine, 0], [0.45 * MAC1, 0, d]) for d in np.linspace(0.1 * b/2, 0.6 * b/2, round(N_hover / 4))]
distrWeight = RunningLoad([[0]*3, [ -w.wing.get_weight()[0] * 9.81 / b ]*3], [0, b/4, b/2], 2)

eql = EquilibriumEquation(kloads=[distrWeight], ukloads=ukloads)
eql.SetupEquation()
Fx, Fy, Fz, Mx, My, Mz = eql.SolveEquation()
box = WingBox(thickness, 0.8 * c_r, 0.8 * 0.17 * c_r, aluminum)

print("AR = ", AR, " b = ", b)
print("VTOL: τ, σ, Y [MPa]")
print(tauVTOL := box.tau(box.b/2, 0, Fx, Fy, Mz)*1e-6)
print(oVTOL := box.o(0, box.h/2, -Mx, My)*1e-6)
print(YVTOL := (oVTOL ** 2 + 3 * tauVTOL ** 2) ** 0.5)

rho = 1.205
toc = 0.17

liftWing = 2 * (9.81 * mTO * nmax * 1.5 / 2) / ( np.pi * b )
zpos = np.linspace(0, b/2, 1000)
Lift = RunningLoad([[0] * len(zpos), [liftWing * ( 1 - (z/b) ** 2 ) ** 0.5 for z in zpos]], zpos, axis=2)
Drag = RunningLoad([[liftWing / LD_ratio * ( 1 - (z/b) ** 2 ) ** 0.5 for z in zpos], [0] * len(zpos)], zpos, axis=2, poa=(-0.25 * MAC1, 0))
ADMoment = Moment([0, 0, Cm_ac_front * 0.5 * rho * V_cruise ** 2 * S_front * MAC1])
Thrusts = [PointLoad([-liftWing * 4 / (LD_ratio*N_cruise), 0, 0],
                     [0.45 * MAC1, toc * MAC1 * 0.5, d]) for d in np.linspace(0.1 * b/2, 0.6 * b/2, round(N_cruise / 4))]

cruise = EquilibriumEquation(kloads=[Lift, Drag, ADMoment, distrWeight] + Thrusts,
                            ukloads=ukloads)
cruise.SetupEquation()
Rfx, Rfy, Rfz, Rmx, Rmy, Rmz = cruise.SolveEquation()
print(f'{Rmz = }')
print("\nCruise: τ, σ, Y [MPa]")
print(taucr := box.tau(-box.b/2, 0, Rfx, Rfy, Rmz)*1e-6)
print(ocr := box.o(box.b/2, box.h/2, -Rmx, Rmy)*1e-6)
print(Ycr := (ocr ** 2 + 3 * taucr ** 2) ** 0.5)
aluminum = Material.load(material='Al 6061', Condition='T6')
print("\nFatigue & Buckling:")
print(fatigueLife := aluminum.ParisFatigueN(-ocr*1e6 + oVTOL*1e6, box.b, 0.375 * 1.2e-3, box.t/2)*1e-6)
print(bucklingStress := aluminum.buckling(box.b, box.t)*1e-6)


$\vec{x} = [F_{1x}, F_{2x}, F_{1y}, F_{2y}, M_{1x}, M_{2x}, M_{1y}, M_{2y}, M_{1z}, M_{2z}]^T$

$\sum F_x = 0 \qquad \sum F_y = 0 \qquad \sum M_x = 0 \qquad \sum M_y = 0 \qquad \sum M_z = 0$

$\phi_1(b/2) - \phi_2(b/2) = 0 \qquad \phi(z) = \int \frac{M_z(z)}{G \cdot J} dz$

$w(z) = \frac{-1}{E \cdot I_{xx}} \int \int M_x(z) dz^2 \qquad 0 = w_2(b/2) - \phi_2(b/2) \cdot l \cdot \cos{\theta} - w_1(b/2) \qquad \frac{d w_1}{dz} - \frac{d w_2}{dz} = 0$

$v(z) = \frac{-1}{E \cdot I_{yy}} \int \int M_y(z) dz^2 \qquad v_1(z) = v_2(z) + \phi_2(b/2) \cdot l \cdot \sin{\theta} \qquad \frac{d v_1}{dz} = \frac{d v_2}{dz}$

In [ ]:
Thrusts = sum([StepFunction([[Max_T_engine, d, 0]]) for d in np.linspace(0.1 * b/2, 0.6 * b/2, round(N_hover / 4))])
wingWeight = StepFunction([[-w.wing.get_weight()[0] * 9.81 / b, 0, 1]])
kvy1, kvy2 = [sum([wingWeight, Thrusts])]*2
theta, l = 15 * pi / 180, 5
aluminum = Material.load(material='Al 6061', Condition='T6')
boxm = WingBox(thickness, 0.8 * MAC1, 0.8 * 0.17 * MAC1, aluminum)
b = (AR * S_front) ** 0.5

F1x, F2x, F1y, F2y = [StepFunction([[1, 0, 0]])]*4
M1x, M2x, M1y, M2y, M1z, M2z = [StepFunction([[1, 0, 0]])]*6

state = np.array([[
    0,
    -(kvy1(b/2) + kvy2(b/2)),
    -(kvy1.integral() + kvy2.integral())(b/2),
    0,
    -Thrusts(b/2) * 0.45 * MAC1 * 2,
    0,
    (Thrusts.integral()(b/2) * 0.45 * MAC1) * l * cos(theta) / (aluminum.G * boxm.J()),
    0,
    -(Thrusts.integral()(b/2) * 0.45 * MAC1) * l * sin(theta) / (aluminum.G * boxm.J()),
    0
]]).T
state

## Single Wing

In [ ]:
config = 3

#sensitivity ------------

#------------------------


WoS = WS
Pmax = 8.77182
mProp = 400 / N_cruise
thickness = 3e-3
w_fus= 1.3; h_fus=1.6; l_fus=4
nmax = 3.02
b = (AR * S_front) ** 0.5

w = Weight(95, Wing(mTO, S_front, S_back, 1.5*nmax, AR, [0.4, 3.6], config),
           Fuselage(mTO, Pmax, l_fus, 5, l_fus/2, config),
          LandingGear(mTO, l_fus/2),
          Propulsion(N_cruise, [mProp]*N_cruise, pos_prop=[3.6]*int(N_cruise/2) + [0.4]*int(N_cruise/2)),
          cargo_m, cargo_p, Bat_mass, l_fus/2, [0.8, 1.3, 1.3, 2.5, 2.5])
aluminum = Material.load(material='Al 6061', Condition='T6')

ukloads = [PointLoad([1, 0, 0], [0, 0, 0]), PointLoad([0, 1, 0], [0, 0, 0]), PointLoad([0, 0, 1], [0, 0, 0]),
          Moment([1, 0, 0]), Moment([0, 1, 0]), Moment([0, 0, 1])]

Thrust = PointLoad([0, Max_T_wing_engine, 0], [0, 0, 0.1 * b])
distrWeight = RunningLoad([[0]*3, [ -w.wing.get_weight()[0] * 9.81 / b ]*3], [0, b/4, b/2], 2)

box = WingBox(thickness, 0.8 * c_r, 0.8 * 0.17 * c_r, aluminum)

eql = EquilibriumEquation(kloads=[distrWeight], ukloads=ukloads)
eql.SetupEquation()
Fx, Fy, Fz, Mx, My, Mz = eql.SolveEquation()
print("AR = ", AR, " b = ", b)

print("VTOL: τ, σ, Y [MPa]")
print(tauVTOL := box.tau(box.b/2, 0, Fx, Fy, Mz)*1e-6)
print(oVTOL := box.o(-box.b/2, -box.h/2, -Mx, My)*1e-6)
print(YVTOL := (oVTOL ** 2 + 3 * tauVTOL ** 2) ** 0.5)

rho = 1.205
toc = 0.17
box = WingBox(thickness, 0.8 * c_r, 0.8 * 0.17 * c_r, aluminum)

liftWing = 2 * (9.81 * mTO * nmax * 1.5) / ( np.pi * b )
zpos = np.linspace(0, b/2, 1000)
Lift = RunningLoad([[0] * len(zpos), [liftWing * ( 1 - (z/b) ** 2 ) ** 0.5 for z in zpos]], zpos, axis=2)
Drag = RunningLoad([[liftWing / LD_ratio * ( 1 - (z/b) ** 2 ) ** 0.5 for z in zpos], [0] * len(zpos)], zpos, axis=2, poa=(-0.25 * MAC1, 0))
ADMoment = Moment([0, 0, Cm_ac_front * 0.5 * rho * V_cruise ** 2 * S_front * MAC1])
wEngine = PointLoad([0, -981, 0], [0, 0, 0.1 * b])
cruise = EquilibriumEquation(kloads=[Lift, Drag, ADMoment, distrWeight, wEngine],
                            ukloads=ukloads)
cruise.SetupEquation()
Rfx, Rfy, Rfz, Rmx, Rmy, Rmz = cruise.SolveEquation()
print("\nCruise: τ, σ, Y [MPa]")
print(taucr := box.tau(-box.b/2, 0, Rfx, Rfy, Rmz)*1e-6)
print(ocr := box.o(-box.b/2, -box.h/2, -Rmx, Rmy)*1e-6)
print(Ycr := (ocr ** 2 + 3 * taucr ** 2) ** 0.5)
aluminum = Material.load(material='Al 6061', Condition='T6')
print("\nFatigue:")
print(fatigueLife := aluminum.ParisFatigueN(ocr*1e6 - oVTOL*1e6, box.b, 0.375 * 1.2e-3, box.t/2)*1e-6)
print(bucklingStress := aluminum.buckling(box.b, box.t)*1e-6)